In [1]:
from utils import *
import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
import os.path
import matplotlib.pyplot as plt
import math
%matplotlib inline

# connect DB

In [2]:
try:
    conn = psycopg2.connect("dbname='mimic' user='mimic' host='localhost' password='meladymimic315'")
except:
    print "I am unable to connect to the database"
print 'connection established'

connection established


# load admissionID 

In [3]:
_adm = np.load('res/admission_ids.npy').tolist()
admission_ids = _adm['admission_ids']
admission_ids_txt = _adm['admission_ids_txt']

# load filtered itemID

In [4]:
v = np.load('res/filtered_input.npy').tolist()
valid_input = v['id']
valid_input_unit = v['unit']

v = np.load('res/filtered_output.npy').tolist()
valid_output = v['id']

v = np.load('res/filtered_chart.npy').tolist()
valid_chart = v['id']
valid_chart_unit = v['unit']

v = np.load('res/filtered_chart_cate.npy').tolist()
valid_chart_cate = v['id']

v = np.load('res/filtered_lab.npy').tolist()
valid_lab = v['id']
valid_lab_unit = v['unit']

v = np.load('res/filtered_lab_num.npy').tolist()
valid_lab_num = v['id']
valid_lab_num_unit = v['unit']

v = np.load('res/filtered_lab_cate.npy').tolist()
valid_lab_cate = v['id']

v = np.load('res/filtered_microbio.npy').tolist()
valid_microbio = v['id']

v = np.load('res/filtered_prescript.npy').tolist()
valid_prescript = v['id']
valid_prescript_unit = v['unit']
allids = valid_input+valid_output+valid_chart+valid_chart_cate+valid_lab+valid_lab_num+valid_lab_cate+valid_microbio+valid_prescript

# list final number of parameters

In [5]:
print 'len(valid_input) = '+ str(len(valid_input))
print 'len(valid_output) = '+ str(len(valid_output))
print 'len(valid_chart) = '+ str(len(valid_chart))
print 'len(valid_chart_cate) = '+ str(len(valid_chart_cate))
print 'len(valid_lab) = '+ str(len(valid_lab))
print 'len(valid_lab_num) = '+ str(len(valid_lab_num))
print 'len(valid_lab_cate) = '+ str(len(valid_lab_cate))
print 'len(valid_microbio) = '+ str(len(valid_microbio))
print 'len(valid_prescript) = '+ str(len(valid_prescript))
print '\nlen(allids) = '+ str(len(allids) )

len(valid_input) = 277
len(valid_output) = 68
len(valid_chart) = 660
len(valid_chart_cate) = 1
len(valid_lab) = 372
len(valid_lab_num) = 54
len(valid_lab_cate) = 231
len(valid_microbio) = 9085
len(valid_prescript) = 2371

len(allids) = 13119


In [6]:
admission_ids_txt =""
for t in admission_ids:
    admission_ids_txt += str(t)+","
admission_ids_txt=admission_ids_txt[0:-1]

valid_input_txt =""
for t in valid_input:
    valid_input_txt += str(t)+","
valid_input_txt=valid_input_txt[0:-1]

valid_output_txt =""
for t in valid_output:
    valid_output_txt += str(t)+","
valid_output_txt=valid_output_txt[0:-1]

valid_chart_txt =""
for t in valid_chart:
    valid_chart_txt += str(t)+","
valid_chart_txt=valid_chart_txt[0:-1]

valid_chart_cate_txt =""
for t in valid_chart_cate:
    valid_chart_cate_txt += str(t)+","
valid_chart_cate_txt=valid_chart_cate_txt[0:-1]

valid_lab_txt =""
for t in valid_lab:
    valid_lab_txt += str(t)+","
valid_lab_txt=valid_lab_txt[0:-1]

valid_lab_num_txt =""
for t in valid_lab_num:
    valid_lab_num_txt += str(t)+","
valid_lab_num_txt=valid_lab_num_txt[0:-1]

valid_lab_cate_txt =""
for t in valid_lab_cate:
    valid_lab_cate_txt += str(t)+","
valid_lab_cate_txt=valid_lab_cate_txt[0:-1]

valid_microbio_txt ="'"
for t in valid_microbio:
    valid_microbio_txt += str(t)+"','"
valid_microbio_txt=valid_microbio_txt[0:-2]

valid_prescript_txt ="'"
for t in valid_prescript:
    valid_prescript_txt += str(t)+"','"
valid_prescript_txt=valid_prescript_txt[0:-2]

# map itemID to index 0 to n
the index here will be a number column of the matrix that we will place the observed value in.

In [ ]:
allitem = valid_output + valid_input + valid_chart + valid_chart_cate + valid_lab + valid_lab_num + valid_lab_cate + valid_microbio + valid_prescript
# len(valid_input) = 277
# len(valid_output) = 68
# len(valid_chart) = 660
# len(valid_chart_cate) = 1
# len(valid_lab) = 372
# len(valid_lab_num) = 54
# len(valid_lab_cate) = 231
# len(valid_microbio) = 9085
# len(valid_prescript) = 2371

f = open('res/summaryVariables.txt','w')

ct = 0
for i in range(len(allitem)):
    if(i<277+68+660+1+372+54+231+9085+2371): s='PRES'
    if(i<277+68+660+1+372+54+231+9085): s='MBIO'
    if(i<277+68+660+1+372+54+231): s='LABC'
    if(i<277+68+660+1+372+54): s='LABN'
    if(i<277+68+660+1+372): s='LAB'
    if(i<277+68+660+1): s='CHARTC'
    if(i<277+68+660): s='CHART'
    if(i<277+68): s='OUT'
    if(i<277): s='IN'
    
    print allitem[i]
    try:
        j = int(allitem[i])
        
        cur = conn.cursor()
        cur.execute('SELECT * FROM mimiciii.d_items WHERE itemid = '+ str(j))
        item = cur.fetchone()
        cur = conn.cursor()
        cur.execute('SELECT * FROM mimiciii.d_labitems WHERE itemid = '+ str(j))
        labitem = cur.fetchone()

        if(item != None):
            f.write(str(ct)+"\t"+ s + "\t"+ str(item[1])+"\t"+ str(item[2])+'\n')
            pass;
        else:
            f.write(str(ct)+"\t"+ s + "\t"+ str(labitem[1])+"\t"+ str(labitem[2]) +'\n')
            pass;

    except:
        #prescriptions
        cur = conn.cursor()
        cur.execute('SELECT formulary_drug_cd, drug FROM mimiciii.prescriptions WHERE formulary_drug_cd = \''+ allitem[i] +'\' limit 1')
        pres = cur.fetchone()
        
        if(pres != None):
            f.write(str(ct)+"\t"+ s + "\t"+ str(pres[0])+"\t"+ str(pres[1]) +'\n')
        
        # micro bio
        try:
            mid = allitem[i]
            mid = mid.replace('(','').replace(')','')
            m = mid.split(',')
            cur = conn.cursor()
            sql = 'SELECT spec_type_desc,org_name,ab_name FROM mimiciii.microbiologyevents WHERE 1=1 '

            if(m[0] == ''):
                sql += ' and spec_itemid is null'
            else:
                sql += ' and spec_itemid = '+m[0]

            if(m[1] == ''):
                sql += ' and org_itemid is null'
            else:
                sql += ' and org_itemid = '+m[1]

            if(m[2] == ''):
                sql += ' and ab_itemid is null'
            else:
                sql += ' and ab_itemid = '+m[2]

            cur.execute(sql+' limit 1')
            mcb = cur.fetchone()
            f.write(str(ct)+"\t"+ s + "\t"+ str(allitem[0])+"\t"+ str(mcb[0]) +"\t"+ str(mcb[1]) +"\t"+ str(mcb[2]) +'\n')
            
        except:
            pass;
        pass;
    f.flush();
    ct+=1

226619
226584
226583
226625
226576
226630
226616
226557
226605
226618
226561
226606
226621
226571
227701
226565
226620
226579
226598
226597
226617
226595
226564
226608
226607
226588
226627
226570
226604
226631
226632
226599
226600
226574
226626
226623
227510
226628
226602
226569
226609
227489
227511
226567
226591
226614
226593
226613
226575
226624
226622
226601
226563
226560
226558
226610
226629
226573
226580
226589
226582
226592
226612
226590
226603
227488
226633
226559
225935
225936
228356
226022
226050
225883
225931
221036
225851
222062
222151
225862
225157
225929
227532
227978
225915
225969
222051
227528
225993
225830
226027
226881
225799
220970
222042
225147
228133
220864
223262
225913
226375
225156
221623
225837
226372
225834
226880
225908
225848
226023
228141
220862
227524
227370
225885
225148
225173
225835
228339
225866
225847
227520
225991
221906
221385
225899
225832
228348
225905
225859
225843
226368
226875
225886
228140
225153
225975
226044
225934
228340
226363
222037
226369

For the categorical feature, Here we give a number for each different value. For example, A, B, C, D, we assign A to 1, B to 2, C to 3, D to 4.